In [1]:
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
from azure.identity import DefaultAzureCredential

In [2]:
sub_id = "b746917e-ceb7-4ae0-81e6-3ccd893cb0de"
rg = "dpv2"
workspace = "dpv2-wks"

ml_client = MLClient(DefaultAzureCredential(), sub_id, rg, workspace)


In [17]:
cmd_job = command(
    code="./src",
    command="python main.py --iris-csv ${{inputs.iris_csv}} --learning-rate ${{inputs.learning_rate}} --boosting ${{inputs.boosting}}",
    experiment_name="iris_exp_single",
    environment="AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu@latest",
    inputs={
        "iris_csv": Input(
            type="uri_file",
            path="https://azuremlexamples.blob.core.windows.net/datasets/iris.csv",
        ),
        # metang: learning_rate doesn't need type. when will we use default type, when not?
        "learning_rate": 0.9,
        "boosting": "gbdt",
    },
    compute="cpu-cluster",
)

In [18]:
returned_job = ml_client.jobs.create_or_update(cmd_job)


In [11]:
returned_job.services["Studio"].endpoint

'https://ml.azure.com/runs/ivory_iron_rh157p2h9f?wsid=/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourcegroups/dpv2/workspaces/dpv2-wks&tid=9c37dc6f-e051-496f-832c-e4fcf894c611'

In [22]:
# metang: primary_metric - the metric must be logged in the user's training script using mlflow.log_metric()
sweep_cmd_job = cmd_job(
    learning_rate=Uniform(min_value=0.01, max_value=0.9),
    boosting=Choice(values=["gbdt","dart"]),
)

In [24]:
sweep_job = sweep_cmd_job.sweep(
    sampling_algorithm="random",
    primary_metric="multi_logloss", #test-multi_logloss",
    goal="Minimize",
)

In [25]:
sweep_job.set_limits(max_total_trials=20, max_concurrent_trials=10, timeout=7200)

In [26]:
# submit the sweep, metang: why the below sample doesn't use ml_client.jobs??
# returned_sweep_job = ml_client.create_or_update(sweep_job)

return_sweep_job = ml_client.jobs.create_or_update(sweep_job)
